In [ ]:
%pip install pip -U -q
%pip install -U boto3

In [ ]:
import os, boto3

filename = 'gpt2_lm_v1.keras'

s3 = boto3.client("s3",
                endpoint_url=os.getenv("AWS_S3_ENDPOINT"),
                aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
                aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"))
bucket = os.getenv("AWS_S3_BUCKET")

if bucket not in [bu["Name"] for bu in s3.list_buckets()["Buckets"]]:
s3.create_bucket(Bucket=bucket)

with open(filename, "rb") as f:
s3.upload_fileobj(f, bucket, f'fraud/{filename}')
print(f'uploaded: {bucket}/fraud/{filename}')